In [1]:
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine    
import re
import sys
from tqdm import tqdm #for progress_apply bar.
tqdm.pandas()

print("Loading data..")
df_ratings = pd.read_table('./data/ml-1M/ratings.dat',sep='::',engine='python', usecols=['userId','movieId','rating'])#pd.read_csv('../data/ml-10M/ratings.csv', usecols=['userId','movieId','rating'])

N_movies = 3952 #max Id of movie
N_users = len(df_ratings['userId'].unique())

print("Creating table[user][movie] = rating..")
ratings = df_ratings.pivot_table(index='userId',columns='movieId',values='rating') #create a table[user][movie] = rating

ratings.replace(np.nan, 0, inplace=True) #replace NaN with 0

print("Applying mean..")
sys.stdout.flush()

ratings = ratings.progress_apply(lambda x: x - ratings.mean(axis = 1)) #substract row mean from all elements: Pearson's corr

Yo! Loading data..
Yo! Loaded. Creating table..
Yo! applying mean..


100%|██████████| 3706/3706 [14:13<00:00,  4.65it/s]


In [7]:
#create a table to compare cosine similarity of ith and jth movie 
item_cosine = np.zeros([N_movies+1, N_movies+1], dtype='f') 

count = 0
total = N_movies*N_movies

for m1 in movies:
    for m2 in movies:
        count += 1
        item_cosine[m1][m2] = cosine(ratings[m1], ratings[m2])
        print(("["+str(round(float(count)*100/total,2))+"]"),end='\r')     #progress tracker ,


[100.00]


In [6]:
np.savez_compressed('./data/item_item_cosine_values.txt', a=item_cosine) #save to a text file